In [1]:
import numpy as np
from scipy.stats import rayleigh
from scipy.special import digamma,erf
import scipy.io as sio
import pandas as pd

In [2]:
def parseMat(data):
    dataSeqs= data['Seqs']  
    mdtype = dataSeqs.dtype
    seqs = {n: dataSeqs[n].flatten() for n in mdtype.names}
    if 'Stats' in data:
        dataStats = data['Stats']  
        mdtype = dataSeqs.dtype  
        stats = {n: dataSeqs[n].flatten() for n in mdtype.names}
        
        return seqs,stats
    return seqs, None

In [3]:
class Model:
    def __init__(self,seqs,K):
            self.N = len(seqs['Time'])
            D= np.zeros(self.N)
            sigma = np.zeros(self.N)
            Tmax = np.zeros(self.N)
            self.K = K
            for i in range(self.N):
                D[i] = np.max(seqs['Mark'][i])
                # actual sigma and Tmax
                sigma[i] = np.nan_to_num((4*(np.std(seqs['Time'][i].flatten(), ddof=1)**5) / (3* len(seqs['Time'][i].flatten())))**0.2)
                Tmax[i] = seqs['Time'][i][-1][0] + np.spacing(1)
                
                #this sigma and Tmax is for syn_data
#                 sigma[i] = np.nan_to_num((4*(np.std(seqs['Time'][i], ddof=1)**5) / (3* len(seqs['Time'][i][0])))**0.2)
#                 Tmax[i] = np.max(seqs['Time'][i]) + np.spacing(1)
#                 print('Tmax[',i,']=',Tmax[i])
#                 print('sigma[',i,']=',sigma[i])
            self.D = np.int(np.max(D))
            Tmax = np.mean(Tmax)
            self.w = np.mean(sigma)
#             print(self.w)
#             print(Tmax)
            self.landmark = self.w* range(np.int(np.ceil(Tmax/self.w))+1) # +1 added
#             self.alpha = np.ones(K)   # why is this np.ones(K) ??
            self.alpha = 1
            M = len(self.landmark)
            self.beta = np.ones((self.D,M,self.K,self.D)) / (M*(self.D**2))
            self.b = np.ones((self.D,K))/self.D
            
            ## for testing purpose
#             l1 = np.zeros(int(self.N/2))
#             l2 = np.ones(int(self.N-int(self.N/2)))
#             label = np.append(l1, l2)
            label = np.round(K * np.random.rand(self.N))
            self.r = np.zeros((self.N,K))
            for k in range(K):
                self.r[label==k,k] = 1
            self.kernel = "gauss"
class Alg:
    def __init__ (self, outer = 8,rho = 0.1,inner = 5,thres = 1e-5,Tmax = []):
        self.outer = outer
        self.rho = rho
        self.inner = inner
        self.thres = thres
        self.Tmax = Tmax

In [4]:
def kernel_int(dt,model):
    distance = np.tile(dt.flatten(),(len(model.landmark),1))
    landmark = np.tile(model.landmark.conj(),(len(dt.flatten()),1)).T
    distance = distance.astype(np.float64)
    landmark = landmark.astype(np.float64)
    distance -= landmark
    G = 0
    if model.kernel == 'gauss':
        G = 0.5 *(erf(distance/((np.sqrt(2))*model.w))
                  + erf(landmark/((np.sqrt(2))*model.w)))
    elif model.kernel == 'exp':
        G = 1 - np.exp(-model.w * (distance-landmark));
        G[G<0] = 0
    return G

In [5]:
def kernel(dt,model):
    distance = np.tile(dt.flatten(),(len(model.landmark),1))
    landmark = np.tile(model.landmark.conj(),(len(dt.flatten()),1)).T
    distance = distance.astype(np.float64)
    landmark = landmark.astype(np.float64)
    distance -= landmark
    g = 0
    if model.kernel == 'gauss':
        g = np.exp(-(distance**2)/(2*(model.w**2)))/(np.sqrt(2*np.pi)*model.w)
    elif model.kernel == 'exp':
        g = model.w * np.exp(-model.w * distance)
        g[g>1] = 0
    return g

In [6]:
def E_log_pi(alpha):
    return digamma(alpha)  - digamma(np.sum(alpha))

In [7]:
def Expectation_DMHP(seqs,model,alg):
    Nk = np.sum(model.r,0)
    alpha = model.alpha + Nk
    Elogpi = E_log_pi(alpha)
#     print(LL)
    EX = np.zeros((model.N,model.K))
    for c in range(model.N):
        time = seqs['Time'][c].flatten()
        event = seqs['Mark'][c].flatten()
        Tstart = seqs['Start'][c].flatten()
        if not alg.Tmax:
            Tstop = seqs['Stop'][c].flatten()
        else:
            Tstop = alg.Tmax
            indt = time < alg.Tmax
            time = time[indt]
            event = event[indt]
        N = len(time)
#         print(Tstop-time)
        G = kernel_int(Tstop-time,model)
#         print(np.sum(G))
        LL = Elogpi
        for i in range(N):
            ui = event[i]
            ti = time[i]
        
            E_lambda_i = (np.sqrt(np.pi/2) * model.b[ui-1,:]).flatten() + np.spacing(1)
            V_lambda_i = ((2 - np.pi/2) * (model.b[ui-1,:]**2)).flatten()
#             print(np.sum(E_lambda_i))
#             print(np.sum(V_lambda_i))
            if i > 0:
                uj = event[0:i].flatten()
                tj = time[0:i].flatten()
#                 print(ti-tj)
                gij = kernel(ti-tj,model)
                auiuj = model.beta[uj-1,:,:,ui-1]
                tiled = np.tile(gij,(1,1,1))
                tiled = np.tile(tiled.T,(1,1,model.K))
                pij = np.multiply(tiled,auiuj)
#                 print(pij.shape)
                tmp = np.sum(pij,axis=(0,1))
                E_lambda_i += tmp.conj().T
                tmp = np.sum(pij**2,axis=(0,1))
#                 print(tmp.shape)
#                 print(tmp)
                V_lambda_i += tmp.conj().T
            LL = LL + np.log(E_lambda_i) - (V_lambda_i/ (2*(E_lambda_i**2)))
#             print(LL)
        
#         print(np.sum(model.b, 0))
        LL -= (Tstop-Tstart).flatten() * (np.sqrt(np.pi/2) * np.sum(model.b, 0))
#         print(LL)  #error
        temp = np.tile(G,(model.K,1,1)).T
        temp2 = np.multiply(temp,np.sum(model.beta[event-1,:,:,:],axis = 3))
        tmp = np.sum(temp2,(0,1))
        LL -= tmp
#         print(LL)  # error
        XX = (LL - np.max(LL))
        EX[c,:] = (np.exp(XX) / np.sum(np.exp(XX)))
    
    model.r = EX

In [8]:
def Maximization_DMHP (seqs,model,alg):
    EX = model.r
    A = model.beta
    mu = np.sqrt(np.pi/2)*model.b
    
#     print('A: ', A.shape)

    for inn in range(alg.inner):
        temp1 = A / model.beta
        temp1[np.isnan(temp1)]  = 0
        temp1[np.isinf(temp1)]  = 0
    
        temp2 = (mu**2) / (2*(model.b**2))
        temp2[np.isnan(temp2)]  = 0
        temp2[np.isinf(temp2)]  = 0
    
        temp3 = np.log(mu)  
        temp3[np.isnan(temp3)]  = 0
        temp3[np.isinf(temp3)]  = 0
        
#         print('tmp1=',np.sum(temp1),', tmp2=',np.sum(temp2),', tmp3=',np.sum(temp3))
        
        NLL = np.sum(temp1) + np.sum(temp2) - np.sum(temp3)  # minus.. temp3
#         print('NLL:', NLL)
    
        MuA = 1 / (model.b**2)
        MuA[np.isinf(MuA)]  = 0
        MuB = 0
        MuC = - np.ones(model.b.shape)
    
        AB = np.zeros(A.shape)
        AA =  1 / (model.beta)
        AA[np.isinf(AA)] = 0
        
    
        for c in range(model.N):
        
            time = seqs['Time'][c].flatten()
            event = seqs['Mark'][c].flatten()
            Tstart = seqs['Start'][c].flatten()
        
            if not alg.Tmax:
                Tstop = seqs['Stop'][c].flatten()
            else:
                Tstop = alg.Tmax
                indt = time < alg.Tmax
                time = time[indt]
                event = event[indt]
            N = len(time)
            G = kernel_int(Tstop-time,model)
#             print(Tstop-time)
#             print('G: ')
#             print(G)
#             print('G end')
        
            TMPAA = np.zeros(A.shape)
            TMPAB = np.zeros(A.shape)
            TMPMuC = np.zeros(mu.shape)
            LL = 0
            for i in range(N):
           
                ui = event[i]
                ti = time[i]
            
                tmp = np.tile(G[:,i],(model.D,model.K,1)).T
            
                TMPAA[ui-1,:,:,:] = TMPAA[ui-1,:,:,:] + tmp ## look at here 
            
                lambdai = mu[ui-1,:] + np.spacing(1)  ## added spacing
                pii = lambdai;
            
                if i > 0:
                    uj = event[0:i].flatten()
                    tj = time[0:i].flatten()
#                     print(ti-tj)
                    gij = kernel(ti-tj,model)
#                     print('gij')
#                     print(gij)
                    auiuj = A[uj-1,:,:,ui-1]  ## A instead of model.beta and ui
                    tiled = np.tile(gij,(1,1,1))
                    tiled = np.tile(tiled.T,(1,1,model.K))
                    pij = np.multiply(tiled,auiuj)
#                     print(np.sum(pij, axis=1))
#                     print(pij.shape)
                
                    tmp = np.sum(pij,(0,1))
#                     print(np.sum(tmp))
                    lambdai += tmp.conj().T  ## lambdai += tmp 
#                     print(np.sum(lambdai))                
                    tmp = np.tile(lambdai,(pij.shape[0],pij.shape[1],1))
                    pij /= tmp
                
                    for j in range(i):
                        uj = event[j]
                        TMPAB[uj-1,:,:,ui-1] -= pij[j,:,:]
                    
                LL += np.log(lambdai) 
#                 print(LL)
            
                pii /= lambdai
                TMPMuC[ui-1,:] -= pii
            
#             print('LL:')
#             print(np.sum(LL))
            LL-= (Tstop - Tstart) * (np.sum(mu, axis=0))
#             print(mu.shape)
#             print(Tstop - Tstart)
#             print(np.sum(mu))
#             print((Tstop - Tstart) * (np.sum(mu, axis=0)))
#             print(np.sum(LL))
            temp = np.tile(G,(model.K,1,1)).T
            temp2 = np.multiply(temp,np.sum(A[event-1,:,:,:],axis = 3))
            tmp = np.sum(temp2,(0,1))
#             print(np.sum(tmp))
            LL -= tmp.conj().T ## changed to conj
#             print(np.sum(LL))
        
            MuB += (Tstop-Tstart) * EX[c,:]
        
            for k in range(model.K):
                AA[:,:,k,:] = AA[:,:,k,:] + (EX[c,k]*TMPAA[:,:,k,:]) 
                AB[:,:,k,:] = AB[:,:,k,:] + (EX[c,k]*TMPAB[:,:,k,:]) 
                MuC[:,k] = MuC[:,k] +  (EX[c,k]*TMPMuC[:,k]) 
#             NLL -= np.sum(EX[c,:]*LL)
#             print(np.sum(LL))
            NLL -= np.dot(EX[c,:],LL)
#             print(NLL)
        MuBB = np.tile(MuB,(model.D,1))
        mutmp = (-MuBB + np.sqrt((MuBB)**2 - 4*(MuA * MuC))) / (2*MuA)
        Atmp = -AB / AA
    
        Atmp[np.isnan(Atmp)] = 0
        mutmp[np.isnan(mutmp)] = 0
        Atmp[np.isinf(Atmp)] = 0
        mutmp[np.isinf(mutmp)] = 0
    
        Err = np.sum(np.abs(A - Atmp)) / np.sum(np.abs(A))  ## removed np.nan_to_num()
        print("Inner: ",inn, " NLL: ", NLL, " Err: ",Err)
        
        A = Atmp
        mu = mutmp
        
#         print(A)
#         print(np.sum(A))
    
        if Err < alg.thres or  inn == alg.inner:
            break
    model.beta = A
    model.b = np.sqrt(2/np.pi) * mu
    return NLL
        

In [9]:
def LearnDMHP(seqs,model,alg):
    NLL = np.zeros(alg.outer)
    for outer in range(alg.outer):
        NLL[outer] = Maximization_DMHP(seqs,model,alg)
        Expectation_DMHP(seqs,model,alg)
        
        print("Outer Iter: ",outer," NLL: ", NLL[outer])

In [10]:
def cluster_purity(K, r, actual_label):
    W = np.argmax(r, axis=1)
    C = np.argmax(actual_label, axis=1)
    print('C:', C)
    print('W:', W)
    N = W.shape[0]
    purity = 0
    
    for k in range(K):
        index_k = np.where(W==k)
        print(index_k)
        print('C-W', C[index_k])
        C_k = C[index_k]
        count = []
        for k in range(K):
            count.append(np.sum(C_k==k))
            print(np.sum(C_k==k))

        purity+=np.max(count)
    
    return purity/N
        

In [11]:
linkedInData = sio.loadmat('Data/LinkedinData.mat')
IPTVData = sio.loadmat('Data/IPTVData.mat')
synData = sio.loadmat('Data/syn_data.mat')
IPTVSeqs,IPTVStats = parseMat(IPTVData)
linkedInSeqs,linedInStats = parseMat(linkedInData)
synSeqs,synStats = parseMat(synData)

In [12]:
model = Model(synSeqs,2)
alg = Alg()
LearnDMHP(synSeqs,model,alg)

Inner:  0  NLL:  12482.974789598797  Err:  0.5624860418893123
Inner:  1  NLL:  9654.347240084333  Err:  0.1927011318702729
Inner:  2  NLL:  9638.664148982398  Err:  0.1717403202231693
Inner:  3  NLL:  9624.648970136825  Err:  0.1480257061793507
Inner:  4  NLL:  9613.301712589117  Err:  0.12345289705477519
Outer Iter:  0  NLL:  9613.301712589117
Inner:  0  NLL:  13005.0625932735  Err:  0.23416521245411498
Inner:  1  NLL:  12842.195669406812  Err:  0.11548126827884196
Inner:  2  NLL:  12831.608007893468  Err:  0.09331178931538488
Inner:  3  NLL:  12824.563388798622  Err:  0.07510387591931204
Inner:  4  NLL:  12819.858486996178  Err:  0.06156624110367171
Outer Iter:  1  NLL:  12819.858486996178
Inner:  0  NLL:  12558.147367153158  Err:  0.08177956356890628
Inner:  1  NLL:  12562.60825531226  Err:  0.11213203182335495
Inner:  2  NLL:  12550.300952119342  Err:  0.08921437691738343
Inner:  3  NLL:  12542.817506173107  Err:  0.07202891190065236
Inner:  4  NLL:  12537.93353377182  Err:  0.0587

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':
/usr/lib/python3.6/site-packages/ipykernel_launcher.py:32: RuntimeWarning: divide by zero encountered in true_divide


Inner:  0  NLL:  12679.190096240378  Err:  0.07897705322755877
Inner:  1  NLL:  12702.53509427795  Err:  0.03385904647072628
Inner:  2  NLL:  12699.947991644456  Err:  0.02766300302910252
Inner:  3  NLL:  12698.004607154213  Err:  0.02592471111397273
Inner:  4  NLL:  12696.425408669975  Err:  0.02497492016386426
Outer Iter:  6  NLL:  12696.425408669975
Inner:  0  NLL:  12734.254155457167  Err:  0.07676244591987764
Inner:  1  NLL:  12765.486062820373  Err:  0.0427707126133176
Inner:  2  NLL:  12762.14738584646  Err:  0.0334985932057056
Inner:  3  NLL:  12759.954474116332  Err:  0.0312236712446091
Inner:  4  NLL:  12758.326758947012  Err:  0.028871803912405205
Outer Iter:  7  NLL:  12758.326758947012


In [13]:
np.argmax(model.r,axis=1)

array([0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0])